In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

24/04/08 20:23:35 WARN Utils: Your hostname, Jessicas-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.6 instead (on interface en0)
24/04/08 20:23:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/08 20:23:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
# Add the file to Spark using SparkFiles
spark.sparkContext.addFile(url)


In [5]:
# Read the CSV file into a DataFrame
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [6]:
#Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("my_table")
print(home_sales_df)


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]


In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
result = spark.sql("SELECT date, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 4 GROUP BY date ORDER BY date")
result.show()


+----------+-------------+
|      date|average_price|
+----------+-------------+
|2019-01-01|     365088.0|
|2019-01-02|    271644.29|
|2019-01-03|    381218.86|
|2019-01-04|    359586.14|
|2019-01-05|    323683.57|
|2019-01-06|    301091.13|
|2019-01-07|     327474.5|
|2019-01-08|    304567.83|
|2019-01-09|     243619.0|
|2019-01-10|     287844.4|
|2019-01-11|    277574.29|
|2019-01-12|     218507.0|
|2019-01-13|    294603.86|
|2019-01-14|    284434.57|
|2019-01-15|     254288.5|
|2019-01-16|     335106.5|
|2019-01-17|    333439.09|
|2019-01-18|    297542.56|
|2019-01-19|    294484.22|
|2019-01-20|    312395.33|
+----------+-------------+
only showing top 20 rows



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
result.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
result.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM my_table WHERE price >= 350000")
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.5972130298614502 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("my_table")

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('my_table')

True

In [13]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 1.811981201171875e-05 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
output_path = "path/to/partitioned_data.parquet"
home_sales_df.write.partitionBy("date_built").parquet(output_path)

In [15]:
# 11. Read the formatted parquet data.
parquet_path = "path/to/partitioned_data.parquet"
parquet_data = spark.read.parquet(parquet_path)
parquet_data

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [16]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_table")

In [19]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Run the query on the cached DataFrame
result_cached = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM my_table WHERE view >= 350000")
result_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|         null|
+-------------+

--- 0.33302998542785645 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("my_table")

In [22]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("my_table")

if is_cached:
    print("The 'my_table'  is still cached.")
else:
    print("The 'my_table'  is no longer cached.")


The 'my_table'  is no longer cached.
